In [1]:
import pandas as pd
import numpy as np
from station import Station
%load_ext autoreload
%autoreload

In [2]:
# read in data files
ride_df = pd.read_csv('data/CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals.csv')
map_df = pd.read_csv('data/CTA_-_System_Information_-_List_of__L__Stops.csv')

In [3]:
morse = Station("Morse", ride_df, map_df)

In [4]:
morse.latitude

42.008362

In [6]:
morse.preprocessed

,stationname,daytype,rides,season
datetime,,,,
2001-01-01,Morse,Sun/Hol,1698,Winter
2001-01-02,Morse,Weekday,3807,Winter
2001-01-03,Morse,Weekday,4047,Winter
2001-01-04,Morse,Weekday,4107,Winter
2001-01-05,Morse,Weekday,4381,Winter
2001-01-06,Morse,Sat,3021,Winter
2001-01-07,Morse,Sun/Hol,2187,Winter
2001-01-08,Morse,Weekday,4411,Winter
2001-01-09,Morse,Weekday,4399,Winter


In [9]:
morse.preprocessed['rides'].mean()

3838.794366197183

In [8]:
{'mean':morse.preprocessed['rides'].mean(), 'std':morse.preprocessed['rides'].std()}

939.7350430490086